In [4]:
import pandas as pd
import jsonlines
import os
from glob import glob

In [6]:
files = glob("../outputs/*/*/prometheus-13b-v1.0/alpaca-eval.json")
dfs = []

def filter_score(x):
    if x["score"] > 0:
        return x["score"]
    if "overall score is" in x["critic"]:
        score = int(x["critic"].split("overall score is")[1].split(".")[0].strip().replace(",","").split(" ", 1)[0])
        return score
    elif "a score of " in x["critic"]:
        score = x["critic"].split("a score of ")[1].split(" ")[0].split(".")[0].split(",")[0].strip()
        if score:
            return int(score)
        else:
            return -1
    else:
        return -1

for file in files:
    df = pd.read_json(file, lines=True)
    df["score"] = df.judge.apply(filter_score)
    dfs.append(df)

df = pd.concat(dfs)
df["output_len"] = df.output.apply(len)
df.shape

(33009, 7)

In [8]:
df[df.score > 0].groupby("generator").agg({
    "score": ["mean", "median", "std", "count"],
})

score                   \
                                                        mean median       std   
generator                                                                       
HuggingFaceH4/mistral-7b-sft-beta                   3.177861    4.0  1.695928   
HuggingFaceH4/zephyr-7b-beta                        4.122981    5.0  1.476526   
alignment-handbook/zephyr-7b-dpo-full               4.414907    5.0  1.259502   
alignment-handbook/zephyr-7b-sft-full               3.063433    4.0  1.692326   
heegyu/0504-DCO-1.1b-max-margin-2e-5@epoch-1        1.938979    1.0  1.483916   
heegyu/0504-DCO-1.1b-max-margin-2e-5@epoch-2        2.132005    1.0  1.625854   
heegyu/0504-DCO-1.1b-max-margin-2e-5@epoch-3        2.202736    1.0  1.684282   
heegyu/0504-DCO-1.1b-max-margin-5e-5@epoch-1        2.092269    1.0  1.603356   
heegyu/0504-DCO-1.1b-max-margin-gamma0.1-5e-5@e...  2.133250    1.0  1.626902   
heegyu/0504-DCO-1.1b-max-margin-gamma0.1-5e-5@e...  2.247512    1.0  1.715705   
heegyu/0504-DCO-1.1b-max-margin-gamma0.1-5e-5@e...  2.220423    1.0  1.703006   
heegyu/0504-mistral-7b-sft-beta-max-margin-1e-4...  3.301863    4.0  1.697836   
heegyu/0504-mistral-7b-sft-beta-max-margin-1e-4...  3.635118    5.0  1.674247   
heegyu/0504-mistral-7b-sft-beta-max-margin-1e-4...  3.730100    5.0  1.645565   
heegyu/0504-mistral-7b-sft-beta-max-margin-5e-5...  3.150498    4.0  1.710997   
heegyu/0504-mistral-7b-sft-beta-max-margin-5e-5...  3.322139    4.0  1.693724   
heegyu/0507-DCO-v2-1.1b-max-margin-gamma0.1-5e-...  2.136816    1.0  1.625969   
heegyu/0507-DCO-v2-1.1b-max-margin-gamma0.1-5e-...  2.366127    1.0  1.769215   
heegyu/0507-DCO-v2-1.1b-max-margin-gamma0.5-5e-...  1.288221    1.0  0.991627   
heegyu/0507-DCO-v2-1.1b-max-margin-gamma0.9-5e-...  1.998756    1.0  1.606726   
heegyu/0507-DCO-v3-1.1b-max-margin-gamma0.1-5e-...  1.903990    1.0  1.560918   
heegyu/0507-DCO-v4-1.1b-max-margin-gamma0.1-5e-...  2.297264    1.0  1.752406   
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...  4.140373    5.0  1.499228   
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-...  3.714286    5.0  1.630053   
heegyu/0509-1.1b-sft-dpo-beta0.01-max-margin@ep...  2.267746    1.0  1.745473   
heegyu/0509-1.1b-sft-dpo-beta0.1-max-margin@epo...  2.000000    1.0  2.000000   
heegyu/0509-DCO-v2-1.1b-max-margin-gamma0.1-5e-...  2.166460    1.0  1.675732   
heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s...  1.905473    1.0  1.412811   
heegyu/test-mistral-7b-sft-beta-lora-max-margin...  3.163975    4.0  1.693695   
heegyu/test-mistral-7b-sft-beta-lora-max-margin...  3.117061    4.0  1.674585   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-1...  2.094645    1.0  1.647490   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-1...  1.631710    1.0  1.380378   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-1...  1.359006    1.0  1.031378   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-5...  2.294776    1.0  1.700171   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-5...  1.931592    1.0  1.582610   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-5...  1.966208    1.0  1.636859   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-5...  1.851806    1.0  1.383626   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-5...  1.903106    1.0  1.478891   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-5...  2.079701    1.0  1.603028   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-5...  1.881841    1.0  1.440291   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-5...  1.848447    1.0  1.381069   
heegyu/test-tinyllama1.1b-sft-beta-max-margin-5...  1.850374    1.0  1.400929   

                                                          
                                                   count  
generator                                                 
HuggingFaceH4/mistral-7b-sft-beta                    804  
HuggingFaceH4/zephyr-7b-beta                         805  
alignment-handbook/zephyr-7b-dpo-full                805  
alignment-handbook/zephyr-7b-sft-full                80

In [ ]:
df[df.score > 0].groupby("generator").agg({
    "score": "mean",
    "output_len": ["mean", "median", "std", "count"],
})

NameError: name 'df' is not defined

# PairRM

In [3]:
files = glob("../outputs/*/*/pairrm/*.json")
dfs = []

for file in files:
    df = pd.read_json(file, lines=True)
    df["dataset"] = os.path.basename(file).split(".")[0]
    dfs.append(df)

df = pd.concat(dfs)
df.shape

(9780, 6)

In [7]:
df.groupby(["dataset", "generator"]).agg({
    "judge": ["mean", "median", "std", "count"],
})

judge  \
                                                                   mean   
dataset    generator                                                      
alpacaeval heegyu/TinyLlama-1.1b-feedback-all@epoch-1         -4.149573   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-2         -4.218166   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-3         -4.611811   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1 -3.949996   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2 -4.393866   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3 -4.331274   
           heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di... -6.657126   
           heegyu/TinyLlama-1.1b-max-margin-0424-orpo@epoch-1 -5.949652   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-1 -4.698814   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-2 -3.780337   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-3 -3.670845   
           heegyu/TinyLlama-1.1b-max-margin-0429-1e-5@epoch-1 -4.336723   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-1 -4.290145   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-2 -4.174094   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-3 -4.410455   
           heegyu/TinyLlama-1.1b-max-margin@epoch-2           -4.185481   
           heegyu/TinyLlama-1.1b-max-margin@epoch-3           -4.247956   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.745288   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.755498   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.836002   
           text_davinci_003                                   -3.869928   

                                                                         \
                                                                 median   
dataset    generator                                                      
alpacaeval heegyu/TinyLlama-1.1b-feedback-all@epoch-1         -4.185547   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-2         -4.335938   
           heegyu/TinyLlama-1.1b-feedback-all@epoch-3         -4.738281   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-1 -4.121094   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-2 -4.478516   
           heegyu/TinyLlama-1.1b-feedback-tree-3-0422@epoch-3 -4.300781   
           heegyu/TinyLlama-1.1b-feedback-tree-3-epoch3-di... -6.960938   
           heegyu/TinyLlama-1.1b-max-margin-0424-orpo@epoch-1 -6.359375   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-1 -4.826172   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-2 -3.875000   
           heegyu/TinyLlama-1.1b-max-margin-0427-orpo@epoch-3 -3.857422   
           heegyu/TinyLlama-1.1b-max-margin-0429-1e-5@epoch-1 -4.525391   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-1 -4.414062   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-2 -4.203125   
           heegyu/TinyLlama-1.1b-max-margin-0429-2e-5@epoch-3 -4.480469   
           heegyu/TinyLlama-1.1b-max-margin@epoch-2           -4.343750   
           heegyu/TinyLlama-1.1b-max-margin@epoch-3           -4.367188   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.880859   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.910156   
           heegyu/TinyLlama__TinyLlama-1.1B-intermediate-s... -3.935547   
           text_davinci_003                                   -4.000000   

                                                                               
                                                                    std count  
dataset    generator                                                           
alpacaeval heegyu/TinyLlama-1.1b-feedback-all@epoch-1          1.350452   276  
           heegyu/TinyLlama-1.1b-feedback-all@epoch-2          1.435997   361  
           heegyu/TinyLlama-1.1b-feedback-all@epoch-3          1